Challenge 2 : BigQueryML - Predicting Emergency Call Response Times

In [1]:
from google.cloud import bigquery
from google.cloud.exceptions import Conflict


In [2]:
client = bigquery.Client()

PROJECT_ID = client.project
DATASET_ID = "emergency_dataset"
LOCATION = "us-central1"

dataset_ref = f"{PROJECT_ID}.{DATASET_ID}"

dataset = bigquery.Dataset(dataset_ref)
dataset.location = LOCATION

try:
    client.create_dataset(dataset, timeout=30)
    print(f"Dataset {DATASET_ID} created.")
except Conflict:
    print(f"Dataset {DATASET_ID} already exists.")



Dataset emergency_dataset already exists.


In [3]:

TABLE_ID = "emergency_calls_response_times"
table_ref = f"{dataset_ref}.{TABLE_ID}"

job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    write_disposition="WRITE_TRUNCATE"
)

file_uri = "gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv"

load_job = client.load_table_from_uri(
    file_uri, table_ref, job_config=job_config
)

load_job.result()
print(f"Successfully loaded {load_job.output_rows} rows into {table_ref}.")

Successfully loaded 50000 rows into qwiklabs-gcp-01-5f9c2ba2a4cd.emergency_dataset.emergency_calls_response_times.


In [4]:
%%bigquery emergency_df
SELECT * FROM emergency_dataset.emergency_calls_response_times

Query is running:   0%|          |

Downloading:   0%|          |

Data Exploration

In [5]:
emergency_df.head(5)

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37


In [6]:
# dataframe: emergency_df
# uuid: ABAD0698-9C7F-47C4-8E9D-8200A0B559E3
# output_variable:
# config_str:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='emergency_df', uuid='ABAD0698-9C7F-47C4-8E9D-8200A0B559E3')

<IPython.core.display.Javascript object>

In [7]:
# dataframe: emergency_df
# uuid: 7D43883E-434D-4523-9F79-5179F8E42F9B
# output_variable:
# config_str:

import google.colabsqlviz.explore_dataframe as _vizcell
_vizcell.explore_dataframe(df_or_df_name='emergency_df', uuid='7D43883E-434D-4523-9F79-5179F8E42F9B')

<IPython.core.display.Javascript object>

In [6]:
emergency_df.dtypes

,0
call_id,Int64
call_timestamp,"datetime64[us, UTC]"
call_type,object
location,object
weather_condition,object
day_of_week,object
time_of_day,Int64
traffic_level,object
distance_to_station,float64
units_available,Int64


In [7]:
emergency_df.head(5)

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37


Feature Engineering

In [8]:
import pandas as pd
import numpy as np

In [9]:
# Convert timestamp and extract Temporal Features
emergency_df['call_timestamp'] = pd.to_datetime(emergency_df['call_timestamp'])
emergency_df['hour_of_day'] = emergency_df['call_timestamp'].dt.hour
emergency_df['day_num'] = emergency_df['call_timestamp'].dt.dayofweek
emergency_df['is_weekend'] = emergency_df['day_num'].apply(lambda x: 1 if x >= 5 else 0)


In [10]:
emergency_df.head()

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time,hour_of_day,day_num,is_weekend
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41,0,6,1
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11,0,6,1
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75,0,6,1
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76,0,6,1
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37,0,6,1


In [11]:
# Create Traffic Interaction Feature
# Mapping traffic to a numerical multiplier
traffic_map = {'Low': 0.8, 'Normal': 1.0, 'High': 1.5, 'Gridlock': 2.0}
emergency_df['traffic_multiplier'] = emergency_df['traffic_level'].map(traffic_map).fillna(1.0)
emergency_df['traffic_weighted_dist'] = emergency_df['distance_to_station'] * emergency_df['traffic_multiplier']

In [12]:
emergency_df.head()

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time,hour_of_day,day_num,is_weekend,traffic_multiplier,traffic_weighted_dist
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41,0,6,1,1.5,32.175
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11,0,6,1,1.5,33.435
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75,0,6,1,1.5,25.785
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76,0,6,1,1.5,26.085
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37,0,6,1,1.5,33.750


In [13]:
# Handle Categorical Encoding (One-Hot Encoding)
categorical_features = ['call_type', 'day_of_week', 'weather_condition', 'location']
emergency_df_encoded = pd.get_dummies(emergency_df, columns=categorical_features, drop_first=True, dtype=int)

In [14]:
emergency_df_encoded.head()

,call_id,call_timestamp,time_of_day,traffic_level,distance_to_station,units_available,response_time,hour_of_day,day_num,is_weekend,...,weather_condition_Snowy,weather_condition_Sunny,weather_condition_Windy,location_Downtown,location_Greenfield,location_Highland,location_Maplewood,location_Oakmont,location_Riverside,location_Uptown
0,35957,2023-01-01 00:05:53+00:00,0,High,21.45,3,23.41,0,6,1,...,0,0,0,0,0,1,0,0,0,0
1,20832,2023-01-01 00:20:47+00:00,0,High,22.29,6,20.11,0,6,1,...,0,0,0,0,0,0,0,1,0,0
2,27949,2023-01-01 00:33:27+00:00,0,High,17.19,14,19.75,0,6,1,...,0,0,1,0,0,0,0,0,1,0
3,20199,2023-01-01 00:48:29+00:00,0,High,17.39,14,20.76,0,6,1,...,0,0,1,0,0,0,0,0,1,0
4,46938,2023-01-01 00:50:44+00:00,0,High,22.50,14,22.37,0,6,1,...,0,1,0,0,0,0,0,0,0,0


In [15]:
# Define Final Model Input Table
# Drop base columns : is_weekend, distance_to_station
final_features = [
    'hour_of_day', 'day_num', 'traffic_weighted_dist',
    'units_available'
]

# Add the newly created dummy columns to the feature list
dummy_cols = [col for col in emergency_df_encoded.columns if any(cat in col for cat in categorical_features)]
final_columns = ['response_time'] + final_features + dummy_cols

# Create the final model input table
emergency_pred_df = emergency_df_encoded[final_columns].dropna()


In [16]:
emergency_pred_df.columns

Index(['response_time', 'hour_of_day', 'day_num', 'traffic_weighted_dist',
       'units_available', 'call_type_Medical', 'call_type_Police',
       'call_type_Rescue', 'day_of_week_Monday', 'day_of_week_Saturday',
       'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday',
       'day_of_week_Wednesday', 'weather_condition_Snowy',
       'weather_condition_Sunny', 'weather_condition_Windy',
       'location_Downtown', 'location_Greenfield', 'location_Highland',
       'location_Maplewood', 'location_Oakmont', 'location_Riverside',
       'location_Uptown'],
      dtype='object')

In [17]:
emergency_pred_df.describe().T

,count,mean,std,min,25%,50%,75%,max
response_time,50000.0,17.446134,5.296881,2.01,13.59,17.25,21.1,36.55
hour_of_day,50000.0,11.5143,6.930766,0.0,6.0,12.0,18.0,23.0
day_num,50000.0,3.01092,1.995946,0.0,1.0,3.0,5.0,6.0
traffic_weighted_dist,50000.0,16.599991,10.796185,0.08,7.81575,15.5575,23.21,45.0
units_available,50000.0,7.9979,4.315867,1.0,4.0,8.0,12.0,15.0
call_type_Medical,50000.0,0.24956,0.432763,0.0,0.0,0.0,0.0,1.0
call_type_Police,50000.0,0.25072,0.433432,0.0,0.0,0.0,1.0,1.0
call_type_Rescue,50000.0,0.24802,0.431868,0.0,0.0,0.0,0.0,1.0
day_of_week_Monday,50000.0,0.14216,0.349218,0.0,0.0,0.0,0.0,1.0
day_of_week_Saturday,50000.0,0.1468,0.35391,0.0,0.0,0.0,0.0,1.0


Model Data Visualization

Save Model Table

In [18]:

DEST_TABLE_ID = 'emergency_training_data'
dest_table_ref =  f"{dataset_ref}.{DEST_TABLE_ID}"


job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

job = client.load_table_from_dataframe(
    emergency_pred_df, dest_table_ref, job_config=job_config
)

# Wait for the job to complete
job.result()

print(f"Successfully saved {len(emergency_pred_df)} rows to {dest_table_ref}")

Successfully saved 50000 rows to qwiklabs-gcp-01-5f9c2ba2a4cd.emergency_dataset.emergency_training_data


Create ML Model

In [ ]:

# Pred :
# 'response_time'
# Vars :
# 'hour_of_day', 'day_num', 'traffic_weighted_dist', 'units_available',
# 'call_type_Medical', 'call_type_Police', 'call_type_Rescue',
# 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
# 'weather_condition_Snowy', 'weather_condition_Sunny', 'weather_condition_Windy',
# 'location_Downtown', 'location_Greenfield', 'location_Highland', 'location_Maplewood', 'location_Oakmont', 'location_Riverside', 'location_Uptown'

In [19]:
model_name = 'emergency_xgb_model'

create_model_sql = f"""
CREATE OR REPLACE MODEL `{dataset_ref}.{model_name}`
OPTIONS(
    MODEL_TYPE='BOOSTED_TREE_REGRESSOR',
    INPUT_LABEL_COLS=['response_time'],
    MAX_ITERATIONS=50,
    EARLY_STOP=TRUE,
    BOOSTER_TYPE='GBTREE',
    LEARN_RATE=0.1
) AS
SELECT *
FROM `{dest_table_ref}`
"""

In [20]:
print(f"Starting XGBoost training for {model_name}...")

query_job = client.query(create_model_sql)
query_job.result()

print(f"Success! Model trained and saved as {dataset_ref}.{model_name}")

Starting XGBoost training for emergency_xgb_model...
Success! Model trained and saved as qwiklabs-gcp-01-5f9c2ba2a4cd.emergency_dataset.emergency_xgb_model


In [21]:
# Model Evaluation
eval_query = f"SELECT * FROM ML.EVALUATE(MODEL `{dataset_ref}.{model_name}`)"
eval_results = client.query(eval_query).to_dataframe()

print("\n--- Model Performance ---")
print(eval_results[['mean_absolute_error', 'mean_squared_error', 'median_absolute_error','r2_score', 'explained_variance']])


--- Model Performance ---
   mean_absolute_error  mean_squared_error  median_absolute_error  r2_score  \
0             1.983591            6.128137               1.707235  0.782068   

   explained_variance  
0            0.797294  


Model Prediction

In [ ]:

# Pred :
# 'response_time'
# Vars :
# 'hour_of_day', 'day_num', 'traffic_weighted_dist', 'units_available',
# 'call_type_Medical', 'call_type_Police', 'call_type_Rescue',
# 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday',
# 'weather_condition_Snowy', 'weather_condition_Sunny', 'weather_condition_Windy',
# 'location_Downtown', 'location_Greenfield', 'location_Highland', 'location_Maplewood', 'location_Oakmont', 'location_Riverside', 'location_Uptown'

In [27]:

test_query = f"""
SELECT *
FROM ML.PREDICT(
  MODEL `{dataset_ref}.{model_name}`,
  (SELECT

        CAST(FLOOR(RAND() * 24) AS INT64) AS hour_of_day,
        CAST(FLOOR(RAND() * 7) AS INT64) AS day_num,
        (RAND() * 15) AS traffic_weighted_dist,
        CAST(FLOOR(RAND() * 10) AS INT64) AS units_available,

        CAST(ROUND(RAND()) AS INT64) AS call_type_Medical,
        CAST(ROUND(RAND()) AS INT64) AS call_type_Police,
        CAST(ROUND(RAND()) AS INT64) AS call_type_Rescue,

        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Monday,
        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Saturday,
        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Sunday,
        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Thursday,
        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Tuesday,
        CAST(ROUND(RAND()) AS INT64) AS day_of_week_Wednesday,

        CAST(ROUND(RAND()) AS INT64) AS weather_condition_Snowy,
        CAST(ROUND(RAND()) AS INT64) AS weather_condition_Sunny,
        CAST(ROUND(RAND()) AS INT64) AS weather_condition_Windy,

        CAST(ROUND(RAND()) AS INT64) AS location_Downtown,
        CAST(ROUND(RAND()) AS INT64) AS location_Greenfield,
        CAST(ROUND(RAND()) AS INT64) AS location_Highland,
        CAST(ROUND(RAND()) AS INT64) AS location_Maplewood,
        CAST(ROUND(RAND()) AS INT64) AS location_Oakmont,
        CAST(ROUND(RAND()) AS INT64) AS location_Riverside,
        CAST(ROUND(RAND()) AS INT64) AS location_Uptown

      FROM UNNEST(GENERATE_ARRAY(1, 10)) AS row_id
  )
)
"""

In [28]:
query_job = client.query(test_query)
results_df = query_job.to_dataframe()

In [29]:
print(results_df.to_string(index=False))

 predicted_response_time  hour_of_day  day_num  traffic_weighted_dist  units_available  call_type_Medical  call_type_Police  call_type_Rescue  day_of_week_Monday  day_of_week_Saturday  day_of_week_Sunday  day_of_week_Thursday  day_of_week_Tuesday  day_of_week_Wednesday  weather_condition_Snowy  weather_condition_Sunny  weather_condition_Windy  location_Downtown  location_Greenfield  location_Highland  location_Maplewood  location_Oakmont  location_Riverside  location_Uptown
               12.161077           10        5              10.516995                8                  1                 0                 0                   1                     1                   1                     1                    0                      1                        0                        1                        1                  1                    1                  0                   0                 0                   0                1
               12.618834           22     